In [154]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
import numpy as np
from minichess.concepts.concepts import in_check, random, threat_opp_queen, material_advantage, has_mate_threat, opponent_has_mate_threat, has_contested_open_file

np.seterr(over="ignore", invalid="raise")

# Your agent's name here
model_name = "gladiator-rn"
# The epochs you want to sample from
agents_to_sample = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# The name of the board-variant here
full_name = "5x4silverman"
dims = (5, 4)

# This can be replaced by some other concept function
CONCEPT_FUNC = material_advantage

concept_name = material_advantage.__name__

epochs_to_look_at = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [156]:
import tensorflow as tf
from minichess.agents.lite_model import LiteModel
from minichess.agents.predictor_convnet import PredictorConvNet

def load_model(full_name, model_name, epoch):
    keras_model = tf.keras.models.load_model("minichess/agents/checkpoints/{}/{}/{}".format(full_name, model_name, epoch))
    simple_model = PredictorConvNet(LiteModel.from_keras_model(keras_model))
    del keras_model
    return simple_model

agents = [load_model(full_name, model_name, epoch) for epoch in agents_to_sample]

INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpr06fzekr\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpr06fzekr\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpj9nb1w5w\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpj9nb1w5w\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpry96o3wa\assets


...

INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmplk4lv2ao\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpv_f2z825\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpv_f2z825\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpc8dbplqc\assets


INFO:tensorflow:Assets written to: C:\Users\Rares\AppData\Local\Temp\tmpc8dbplqc\assets


In [157]:
from minichess.agents.lite_model import LiteModel
from minichess.agents.predictor_convnet import PredictorConvNet
from minichess.chess.fastchess import Chess
from minichess.chess.fastchess_utils import piece_matrix_to_legal_moves, visualize_board
from minichess.chess.move_utils import calculate_all_moves, index_to_move, move_to_index
from minichess.rl.chess_helpers import get_initial_chess_object
import numpy as np

import tensorflow as tf
def play_match(agents, full_name, dims, move_cap, all_moves, all_moves_inv, concept_function):
    chess = get_initial_chess_object(full_name)
    to_start = 1 if np.random.random() > 0.5 else 0
    current = to_start
    positive_cases = []
    negative_cases = []
    SAMPLING_RATIO = 0.2

    while chess.game_result() is None:
        if np.random.random() < SAMPLING_RATIO:
            if concept_function(chess):
                positive_cases.append(chess.agent_board_state())
            else:
                negative_cases.append(chess.agent_board_state())


        agent_to_play = agents[current]
        dist, value = agent_to_play.predict(chess.agent_board_state())

        moves, proms = chess.legal_moves()
        legal_moves = piece_matrix_to_legal_moves(moves, proms)
        legal_moves_mask = np.zeros((dims[0], dims[1], all_moves_inv.shape[0]))
        for move in legal_moves:
            (i, j), (dx, dy), promotion = move
            ind = move_to_index(all_moves, dx, dy, promotion, chess.turn)
            legal_moves_mask[i, j, ind] = 1

        move_dims = dist.shape

        dist = (dist + 0.5 * np.random.uniform(size=dist.shape)) * legal_moves_mask.flatten()

        dist /= dist.sum()
        move_to_play = np.argmax(dist)

        # move_to_play = np.random.choice(np.arange(dist.shape[0]), p=dist)
        i, j, ind = np.unravel_index(move_to_play, (dims[0], dims[1], move_cap))
        dx, dy, promotion = index_to_move(all_moves_inv, ind, chess.turn)
        chess.make_move(i, j, dx, dy, promotion)
        current = (current + 1) % 2
    return positive_cases, negative_cases

In [158]:
all_moves, all_moves_inv = calculate_all_moves(dims)
move_cap = all_moves_inv.shape[0]

In [159]:
from tqdm import tqdm
positive_cases = []
negative_cases = []

CASES_TO_COLLECT = 2000
pbar = tqdm(total=CASES_TO_COLLECT)
while len(positive_cases) < CASES_TO_COLLECT:
    pos, neg = play_match([agents[0], agents[2]], full_name, dims, move_cap, all_moves, all_moves_inv, CONCEPT_FUNC)
    positive_cases.extend(pos)
    negative_cases.extend(neg)
    pbar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_COLLECT]
negative_cases = negative_cases[:CASES_TO_COLLECT]

2001it [1:55:54,  3.48s/it]


In [160]:
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [161]:
positive_cases.shape[0]

2000

In [162]:
negative_cases.shape[0]

2000

In [163]:
from minichess.agents.convnet import ConvNet

for epoch_to_look_at in epochs_to_look_at:
    import tensorflow.keras as keras
    predictor_model = ConvNet(None, None, init=False)
    predictor_model.model = keras.models.load_model("minichess/agents/checkpoints/{}/{}/{}".format(full_name, model_name, epoch_to_look_at))
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])
    np.random.shuffle(shuffled_indices)
    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]
    POSITIONS_TO_CONSIDER = 3200
    VALIDATION_POSITIONS = 800
    from minichess.concepts.linear_regression import perform_linear_regression, perform_logistic_regression, perform_regression

    concept_presences = {}


    print("Getting outputs...")
    outputs = predictor_model.get_all_resblock_outputs(all_cases)
    # actual_outputs = predictor_model.predict(boards, id_vector_to_use)
    # Outputs blir returnert i batcher, må flette det sammen
    print("Merging outputs...")
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    outputs = merged_outputs
    print("Outputs merged.")
    # Aktiveringer fra res-block i
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        print(points.shape)
        # Så man har (n, k) sampler der n er antallet posisjoner, og k er det totale antallet aktiveringsverdier i lag i.
        print("Performing regression for layer {}".format(i))
        # points = np.concatenate([points, actual_outputs], axis=1)
        score = perform_regression(
            points[:POSITIONS_TO_CONSIDER],
            all_labels[:POSITIONS_TO_CONSIDER],
            points[POSITIONS_TO_CONSIDER:],
            all_labels[POSITIONS_TO_CONSIDER:],
            True
        )
        concept_presence_per_layer.append(score)

        print("The presence of {} in resblock {} is {}".format(concept_name, i, score))
    concept_presences[concept_name] = concept_presence_per_layer
    import os
    import string
    from random import choices
    import json

    os.makedirs("concept_presences", exist_ok=True)
    os.makedirs("concept_presences/{}".format(full_name), exist_ok=True)
    os.makedirs("concept_presences/{}/{}".format(full_name, model_name), exist_ok=True)
    os.makedirs("concept_presences/{}/{}/{}".format(full_name, model_name, concept_name), exist_ok=True)
    os.makedirs("concept_presences/{}/{}/{}/{}".format(full_name, model_name, concept_name, epoch_to_look_at), exist_ok=True)

    random_suffix = ''.join(choices(string.ascii_uppercase + string.digits, k=10))

    with open("concept_presences/{}/{}/{}/{}/{}.json".format(full_name, model_name, concept_name, epoch_to_look_at, random_suffix), "w") as f:
        json.dump(concept_presences[concept_name], f)


Getting outputs...


100%|██████████| 125/125 [00:01<00:00, 109.57it/s]

Merging outputs...
Outputs merged.
(4000, 1280)
Performing regression for layer 0
Epoch 1/50


100/100 [==============================] - 0s 2ms/step - loss: 1.0934 - val_loss: 1.0514
Epoch 2/50
100/100 [==============================] - 0s 972us/step - loss: 1.0092 - val_loss: 0.9727
Epoch 3/50
100/100 [==============================] - 0s 979us/step - loss: 0.9350 - val_loss: 0.9040
Epoch 4/50
100/100 [==============================] - 0s 984us/step - loss: 0.8712 - val_loss: 0.8455
Epoch 5/50
100/100 [==============================] - 0s 928us/step - loss: 0.8177 - val_loss: 0.7975
Epoch 6/50
100/100 [==============================] - 0s 970us/step - loss: 0.7746 - val_loss: 0.7595
Epoch 7/50
100/100 [==============================] - 0s 936us/step - loss: 0.7418 - val_loss: 0.7321
Epoch 8/50
100/100 [==============================] - 0s 937us/step - loss: 0.7195 - val_loss: 0.7151
Epoch 9/50
100/100 [==============================] - 0s 938us/step - loss: 0.7076 - val_loss: 0.7068
Epoch 10/50
100/100 [==============================] - 0s 929us/step - loss: 0.7018 - val_loss:

100/100 [==============================] - 0s 918us/step - loss: 0.6936 - val_loss: 0.6939
Epoch 23/50
100/100 [==============================] - 0s 943us/step - loss: 0.6936 - val_loss: 0.6939
Epoch 24/50
100/100 [==============================] - 0s 912us/step - loss: 0.6937 - val_loss: 0.6942
Epoch 25/50
100/100 [==============================] - 0s 902us/step - loss: 0.6938 - val_loss: 0.6940
Epoch 26/50
100/100 [==============================] - 0s 907us/step - loss: 0.6937 - val_loss: 0.6942
Epoch 27/50
100/100 [==============================] - 0s 880us/step - loss: 0.6936 - val_loss: 0.6940
Epoch 28/50
100/100 [==============================] - 0s 918us/step - loss: 0.6935 - val_loss: 0.6939
Epoch 29/50
100/100 [==============================] - 0s 883us/step - loss: 0.6937 - val_loss: 0.6941
Epoch 30/50
100/100 [==============================] - 0s 908us/step - loss: 0.6938 - val_loss: 0.6939
Epoch 31/50
100/100 [==============================] - 0s 910us/step - loss: 0.6935 -

...

Merging outputs...
Outputs merged.
(4000, 1280)
Performing regression for layer 0
Epoch 1/50
100/100 [==============================] - 0s 2ms/step - loss: 1.1063 - val_loss: 1.0683
Epoch 2/50
100/100 [==============================] - 0s 944us/step - loss: 1.0449 - val_loss: 1.0153
Epoch 3/50
100/100 [==============================] - 0s 943us/step - loss: 0.9926 - val_loss: 0.9669
Epoch 4/50
100/100 [==============================] - 0s 990us/step - loss: 0.9455 - val_loss: 0.9234
Epoch 5/50
100/100 [==============================] - 0s 946us/step - loss: 0.9031 - val_loss: 0.8844
Epoch 6/50
100/100 [==============================] - 0s 918us/step - loss: 0.8648 - val_loss: 0.8491
Epoch 7/50
100/100 [==============================] - 0s 1ms/step - loss: 0.8309 - val_loss: 0.8180
Epoch 8/50
100/100 [==============================] - 0s 914us/step - loss: 0.8015 - val_loss: 0.7912
Epoch 9/50
100/100 [==============================] - 0s 950us/step - loss: 0.7760 - val_loss: 0.7681
Epoc

100%|██████████| 125/125 [00:01<00:00, 106.79it/s]


Merging outputs...
Outputs merged.
(4000, 1280)
Performing regression for layer 0
Epoch 1/50
100/100 [==============================] - 0s 2ms/step - loss: 1.1196 - val_loss: 1.0849
Epoch 2/50
100/100 [==============================] - 0s 953us/step - loss: 1.0618 - val_loss: 1.0323
Epoch 3/50
100/100 [==============================] - 0s 937us/step - loss: 1.0095 - val_loss: 0.9852
Epoch 4/50
100/100 [==============================] - 0s 954us/step - loss: 0.9626 - val_loss: 0.9426
Epoch 5/50
100/100 [==============================] - 0s 960us/step - loss: 0.9211 - val_loss: 0.9040
Epoch 6/50
100/100 [==============================] - 0s 946us/step - loss: 0.8828 - val_loss: 0.8688
Epoch 7/50
100/100 [==============================] - 0s 954us/step - loss: 0.8491 - val_loss: 0.8380
Epoch 8/50
100/100 [==============================] - 0s 984us/step - loss: 0.8184 - val_loss: 0.8097
Epoch 9/50
100/100 [==============================] - 0s 946us/step - loss: 0.7915 - val_loss: 0.7852
Ep

100%|██████████| 125/125 [00:01<00:00, 113.59it/s]


Merging outputs...
Outputs merged.
(4000, 1280)
Performing regression for layer 0
Epoch 1/50
100/100 [==============================] - 0s 1ms/step - loss: 1.1198 - val_loss: 1.0993
Epoch 2/50
100/100 [==============================] - 0s 839us/step - loss: 1.0602 - val_loss: 1.0454
Epoch 3/50
100/100 [==============================] - 0s 844us/step - loss: 1.0100 - val_loss: 0.9992
Epoch 4/50
100/100 [==============================] - 0s 843us/step - loss: 0.9670 - val_loss: 0.9587
Epoch 5/50
100/100 [==============================] - 0s 831us/step - loss: 0.9284 - val_loss: 0.9220
Epoch 6/50
100/100 [==============================] - 0s 845us/step - loss: 0.8936 - val_loss: 0.8887
Epoch 7/50
100/100 [==============================] - 0s 853us/step - loss: 0.8619 - val_loss: 0.8582
Epoch 8/50
100/100 [==============================] - 0s 843us/step - loss: 0.8333 - val_loss: 0.8313
Epoch 9/50
100/100 [==============================] - 0s 850us/step - loss: 0.8079 - val_loss: 0.8072
Ep